# Demo Setup

In [1]:
import connect
import requests
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet
from automates.gromet.query import query


The following assumes that you have already generated the json (e.g., by running the notebook script `py_src_to_grometFN_JSON.ipynb`)

We first set up access to the MIRA Epidemiology Domain Knowledge Graph (DKG) web service.

In [2]:
MIRA_DKG_URL = 'http://34.230.33.149:8771'

def get_mira_dkg_term(term, attribs):
    res = requests.get(MIRA_DKG_URL + '/api/search', params={'q': term})
    term = [entity for entity in res.json() if entity['id'].startswith('askemo')][0]
    res = {attrib: term.get(attrib) for attrib in attribs if term.get(attrib) is not None}
    return res

# Load CHIME SIR gromet representation

In [3]:
gromet_fn_module = json_to_gromet("gromet/CHIME_SIR_while_loop--Gromet-FN-auto.json")

The following uses the simple query to collect all named output ports of a GrometFNModule.
`nops` will hold a `List` of `Tuples`, where each tuple has the following format:
```
(<name_of_output_port>,    # named output port == a variable in source code
 <literal_value>,          # IF a literal value has been assigned, otherwise None
 <source_code_reference>)  # metadatum for the source code location of the assignment
```

In [4]:
nops = query.collect_named_output_ports(gromet_fn_module)
print(nops[0])

('inv_contact_rate', None, {'code_file_reference_uid': '09a64930-e9e4-8ae5-f825-bdf7c1e729f9',
 'col_begin': 4,
 'col_end': 50,
 'line_begin': 3,
 'line_end': 3,
 'metadata_type': 'source_code_reference',
 'provenance': {'method': 'skema_code2fn_program_analysis',
                'timestamp': '2022-11-30 16:18:33.012848'}})


# Load all the source, document, and other files we need

In [49]:
from IPython.display import IFrame

####
# HERE IS CONTENT FOR PART 2: FORMULA-TO-MODEL-MATCHING
####
# CHIME SVIIR SOURCE
CHIME_SVIIvR_model_source = connect.read_text_from_file("model/SVIIvR/CHIME_SVIIvR.py")

# Formula LaTeX source
CHIME_SVIIvR_FORMULAS_PDF_DOC = "model/SVIIvR/CHIME_SVIIvR_model_equations.pdf"
#formula_pdf_file = "model/SV"
CHIME_SVIIvR_FORMULAS_LATEX = connect.read_text_from_file("model/SVIIvR/formula.tex_idx")


####
# HERE IS CONTENT FOR PART 3: TEXT-TO-MODEL-MATCHING
####
CHIME_SIR_model_source = connect.read_text_from_file("model/SIR/CHIME_SIR_while_loop.py")
CHIME_SIR_DESCRIPTION_PDF_DOC = "model/SIR/CHIME-online-manual-T2021-01-19.pdf";
CHIME_SIR_DESCRIPTION_TEXT = connect.read_text_from_file(connect.index_text("model/SIR/description.txt"))

###
# HERE IS CONTENT FOR PART 4: BUCKY/DATA
###
BUCKY_model_source = connect.read_text_from_file("./model/Bucky/bucky_sample.py")



# Load Ontology 

Next, we choose a set of terms and attributes to pull into a local ontology that we can use to connect to the model. In addition, for some of the terms, we define custom ranges in which their values for this model can be adjusted.

In [5]:
# Terms we want to find in MIRA and specific attributes we want to add to our local ontology
terms = ['population', 'doubling time', 'recovery time', 'infectious time']
attribs = ['description', 'synonyms', 'xrefs', 'suggested_unit', 'suggested_data_type',
           'physical_min', 'physical_max', 'typical_min', 'typical_max']

# The local ontology if filled up from the MIRA DKG
LOCAL_ONTOLOGY = {
    term: get_mira_dkg_term(term, attribs) for term in terms
    }

# We can also set further local / use-case specific constraints as needed
LOCAL_ONTOLOGY['population']['typical_min'] = 1000
LOCAL_ONTOLOGY['population']['typical_max'] = 40_000_000

LOCAL_ONTOLOGY

{'population': {'description': 'The number of people who live in an area being modeled.',
  'synonyms': [],
  'xrefs': [{'id': 'ido:0000509', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'person',
  'suggested_data_type': 'int',
  'physical_min': 0.0,
  'typical_min': 1000,
  'typical_max': 40000000},
 'doubling time': {'description': 'The length of time that an infectious disease requires to double in incidence.',
  'synonyms': [{'value': 'doubling rate',
    'type': 'oboInOwl:hasRelatedSynonym'}],
  'xrefs': [{'id': 'cemo:epidemic_doubling_time', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'recovery time': {'description': 'The length of time an infected individual needs to recover after being infected.',
  'synonyms': [{'value': 'mean recovery time',
    'type': 'oboInOwl:hasExactSynonym'}],
  'xrefs': [],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'infectious time': 

# DEMO: Model-to-Resource Matching

# Demo Part 1: Ontology Matching

### 1a. Ontology to Gromet matching

In [6]:
targets = ['population', 'infectious time']
terms = list(LOCAL_ONTOLOGY.keys())

parameters = set()
var_dict = {}
for nop in nops:
    if nop[1] is not None:
        parameters.add(nop[0])
        var_dict[nop[0]] = nop

discoveredParameterConnections = connect.match_gromet_targets(targets, list(parameters), var_dict, terms)
discoveredParameterConnections

[('population', {'s_n': 'grometSubObject'}, 1000.0, 81),
 ('infectious time', {'infections_days': 'grometSubObject'}, 14.0, 68)]

### 1b. Ontology to code matching

In [7]:
code = "model/SIR/CHIME_SIR_while_loop.py"
targets = ['population', 'infectious time']
discoveredParameterConnections = []
try:
    discoveredParameterConnections = connect.match_code_targets(targets, code, terms)
except OpenAIError as err:
    print("OpenAI connection error:", err)
    print("Using hard-coded connections")
    discoveredParameterConnections = [("infectious time", {"name": "grometSubObject"}, 14.0, 67),("population", {"name": "grometSubObject"}, 1000, 80)]

discoveredParameterConnections

Extracted variables:  [(3, 'inv_contact_rate', '1.0'), (11, 'growth_rate', '0'), (13, 'growth_rate', '2.0'), (34, 'index', '0'), (35, 'p_idx', '0'), (41, 'd_idx', '0'), (64, 'i_day', '17.0'), (65, 'n_days', '20'), (66, 'N_p', '3'), (67, 'N_t', '121'), (68, 'infections_days', '14.0'), (69, 'relative_contact_rate', '0.05'), (70, 'gamma', '1.0'), (81, 's_n', '1000'), (82, 'i_n', '1'), (83, 'r_n', '1'), (85, 'p_idx', '0')]


[('population', {'s_n': 'grometSubObject'}, 1000.0, 81),
 ('infectious time', {'infections_days': 'grometSubObject'}, 14.0, 68)]

# Demo Part 2: Formula-to-Model Matching

## We need the CHIME SVIIvR model...

In [34]:
print(CHIME_SVIIvR_model_source)


import sys
from csv import DictWriter, QUOTE_NONNUMERIC

def get_beta(intrinsic_growth_rate, gamma, susceptible, relative_contact_rate):
    """
    Calculates a rate of exposure given an intrinsic growth rate for COVID-19
    :param intrinsic_growth_rate: Rate of spread of COVID-19 cases
    :param gamma: The expected recovery rate from COVID-19 for infected individuals
    :param susceptible: Current amount of individuals that are susceptible
    :param relative_contact_rate: The relative contact rate amongst individuals in the population
    :return: beta: The rate of exposure of individuals to persons infected with COVID-19
    """
    inv_contact_rate = 1.0 - relative_contact_rate  # The inverse rate of contact between individuals in the population ## get_beta_icr_exp
    updated_growth_rate = intrinsic_growth_rate + gamma  # The intrinsic growth rate adjusted for the recovery rate from infection ## get_beta_ugr_exp
    beta = updated_growth_rate / susceptible * inv_contact_rate  

## And a document with some descriptive formulas... 

In [35]:
IFrame(CHIME_SVIIvR_FORMULAS_PDF_DOC, width=600, height=300)


## We used a tool to extract the source LaTeX from the PDF...

In [36]:
print(CHIME_SVIIvR_FORMULAS_LATEX)


1	S^\prime &=& - \beta S I - \beta S I_{v} - v_{r} S
2	V^\prime &=& v_{r} S - v_{s} V I - v_{s} V I_{v}
3	I^\prime &=& \beta S I + \beta S I_{v} - \gamma I
4	I_{v}^\prime &=& v_{s} V I + v_{s} V I_{v} - \gamma I_{v}
5	R^\prime &=& \gamma I + \gamma I_v



## Finally, we can find connections between the model and its descriptive formulas

In [33]:
# REMINDER: CHANGE THIS SO THE API TAKES IN THE ACTUAL VALUES, NOT THE PATHS
connect.formula_code_connection(CHIME_SVIIvR_model_source, formula)

OSError: [Errno 63] File name too long: 'import sys\nfrom csv import DictWriter, QUOTE_NONNUMERIC\n\ndef get_beta(intrinsic_growth_rate, gamma, susceptible, relative_contact_rate):\n    """\n    Calculates a rate of exposure given an intrinsic growth rate for COVID-19\n    :param intrinsic_growth_rate: Rate of spread of COVID-19 cases\n    :param gamma: The expected recovery rate from COVID-19 for infected individuals\n    :param susceptible: Current amount of individuals that are susceptible\n    :param relative_contact_rate: The relative contact rate amongst individuals in the population\n    :return: beta: The rate of exposure of individuals to persons infected with COVID-19\n    """\n    inv_contact_rate = 1.0 - relative_contact_rate  # The inverse rate of contact between individuals in the population ## get_beta_icr_exp\n    updated_growth_rate = intrinsic_growth_rate + gamma  # The intrinsic growth rate adjusted for the recovery rate from infection ## get_beta_ugr_exp\n    beta =

# Demo Part 3: Text-to-Model Matching

## We need the CHIME SIR model...


In [41]:
print(CHIME_SIR_model_source)

def get_beta(intrinsic_growth_rate, gamma,           
             susceptible, relative_contact_rate):    
    inv_contact_rate = 1.0 - relative_contact_rate  
    updated_growth_rate = intrinsic_growth_rate + gamma  
    beta = updated_growth_rate / susceptible * inv_contact_rate 
 
    return beta  

def get_growth_rate(doubling_time): 
    if doubling_time == 0:  
        growth_rate = 0  
    else:
        growth_rate = 2.0 ** (1.0 / doubling_time) - 1.0 
    return growth_rate 

def sir(s, i, r, beta, gamma, n): 
    s_n = (-beta * s * i) + s  
    i_n = (beta * s * i - gamma * i) + i 
    r_n = gamma * i + r  

    scale = n / (s_n + i_n + r_n) 

    s = s_n * scale 
    i = i_n * scale 
    r = r_n * scale 
    return s, i, r  

def sim_sir(s, i, r, gamma, i_day, 
            N_p, betas, days, 
            d_a, s_a, i_a, r_a, e_a  
            ):
    n = s + i + r 
    d = i_day 
    index = 0  
    p_idx = 0 

    while p_idx < N_p:  
        beta = betas[p_idx]  
        n_da

## And a PDF that describes the SIR model...

In [47]:
IFrame(CHIME_SIR_DESCRIPTION_PDF_DOC, width=600, height=300)


## We use a tool to extract the textual description of the SIR model from a few pages of the doc...

In [48]:
print(CHIME_SIR_DESCRIPTION_TEXT)

0	Discrete-time SIR modeling of infections/recovery
1	The model consists of individuals who are either Susceptible (S), Infected (I), or Recovered (R).
2	The epidemic proceeds via a growth and decline process. This is the core model of infectious disease spread and has been in use in epidemiology for many years.
3	The dynamics are given by the following 3 equations.
4	St+1 = St−βStIt
5	It+1 =It +βStIt−γIt
6	Rt+1 = Rt + γIt
7	To project the expected impact to Penn Medicine, we estimate the terms of the model.
8	To do this, we use a combination of estimates from other locations, informed estimates based on logical reasoning, and best guesses from the American Hospital Association.
9	Parameters
10	The model's parameters, β and γ , determine the severity of the epidemic. β can be interpreted as the effective contact rate: β=τ×c
11	which is the transmissibility τ multiplied by the average number of people exposed c. The transmissibility is the basic virulence of the pathogen. The number of 

## Finally, we can find connections between the model and descriptions from the text

In [13]:
# REMINDER: CHANGE THE API HERE!!!!
connect.code_text_connection(code, text)

Best description for python function get_growth_rate is in lines 25-25:
	24	To estimate β directly, we'd need to know transmissibility and social contact rates. Since we don't know these things, we can extract it from known doubling times. The AHA says to expect a doubling time Td of 7-10 days. That means an early-phase rate of growth can be computed by using the doubling time formula:
>>	25	g = 21/Td −1
	26	Since the rate of new infections in the SIR model is g = βS − γ and we've already computed γ , β becomes
---------------------------------------
Best description for python function get_beta is in lines 11-12:
	10	The model's parameters, β and γ , determine the severity of the epidemic. β can be interpreted as the effective contact rate: β=τ×c
>>	11	which is the transmissibility τ multiplied by the average number of people exposed c. The transmissibility is the basic virulence of the pathogen. The number of people exposed c is the parameter that can be changed through social distan

# Demo Part 4: Data-to-Model Matching

## The Bucky model is large. Imagine the user has chosen some interesting functions...

In [50]:
print(BUCKY_model_source)

def estimate_cfr(
    g_data,
    base_CFR,
    case_to_death_time,
    Rh_gamma_k,
    S_age_dist,
    days_back=7,
):
    """Estimate CFR from recent case data."""

    mean = case_to_death_time  # params["H_TIME"] + params["I_TO_H_TIME"] #+ params["D_REPORT_TIME"]
    adm2_mean = xp.sum(S_age_dist * mean[..., None], axis=0)
    k = Rh_gamma_k

    rolling_case_hist = g_data.csse_data.incident_cases
    rolling_death_hist = g_data.csse_data.incident_deaths

    t_max = rolling_case_hist.shape[0]
    x = xp.arange(0.0, t_max)

    # adm0
    adm0_inc_cases = xp.sum(rolling_case_hist, axis=1)
    adm0_inc_deaths = xp.sum(rolling_death_hist, axis=1)

    adm0_theta = xp.sum(adm2_mean * g_data.Nj / g_data.N) / k

    w = 1.0 / (xp.special.gamma(k) * adm0_theta**k) * x ** (k - 1) * xp.exp(-x / adm0_theta)
    w = w / (1.0 - w)
    w = w / xp.sum(w)
    w = w[::-1]

    # n_loc = rolling_case_hist.shape[1]
    cfr = xp.empty((days_back,))
    for i in range(days_back):
        d = i + 1
  

## And maintains some interesting relevant datasets...

In [15]:
# REMINDER: Let's change this so that in the preamble part up above, we load each of these datasets into a DF.
# We can then list the names of the dataframes, and show the schema plus the first two rows of each one.

dataset_dir =  "./model/Bucky/data_sample/"
connect.print_list(dataset_dir)

ls  ./model/Bucky/data_sample/
	 covid_deaths_usafacts.csv
	 nychealth.csv
	 covid_tracking.csv
	 usafacts_hist.csv
	 covid_confirmed_usafacts.csv


## Finally, we can find connections between the functions and columns in these datasets

In [53]:
# REMINDER: Fix this so it uses improved variable names.
# Ideally, the output would be a bit nicer but not absolutely crucial

connect.code_dataset_connection(code,dataset_dir)

OpenAI connection error: This model's maximum context length is 4097 tokens, however you requested 4754 tokens (4498 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.
